In [150]:
import warnings

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#for visualizations
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


In [151]:
df_train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

#creating copies of both for future use
df_tr=df_train.copy()
df_te=df_test.copy()

#.13231

In [152]:
y=df_tr['SalePrice']


#dropping Id column
df_tr=df_tr.drop(['Id'],axis=1)
df_te=df_te.drop('Id',axis=1)

In [153]:
#missing values % in each column
def missing(df):
    dict_missing_perc={}
    dict_missing_count={}
    for column in df.columns:
        dict_missing_perc[column]=round((df[column].isna().sum()*100/df.shape[0]),2)
        dict_missing_count[column]=df[column].isna().sum()
    sorted_perc=sorted(list(dict_missing_perc.values()),reverse=True)
    sorted_count=sorted(list(dict_missing_count.values()),reverse=True)
   
  
    data_frame=pd.DataFrame({"Column_name":list(dict_missing_perc.keys()),
                             '% missing values':list(dict_missing_perc.values()),
                             'missing_value_count':list(dict_missing_count.values())})
    #data_frame.index=list(dict_missing_perc.keys())
    data_frame.sort_values(by='% missing values',axis=0,ascending=False,inplace=True)
    data_frame=data_frame.style.background_gradient(cmap='rainbow_r')
    
    return data_frame
    



# missing values in training set

In [154]:
df_te['Foundation'].isna().sum()/df_te.shape[0]

In [155]:
pd.set_option('max_rows',100)
missing(df_tr)

# missing values in test set

In [156]:
missing(df_te)

# EDA (Dealing with missing values and skewness)

In [157]:
df_tr['LotArea'].isna().sum()

In [158]:
#creating new categorical features
#There are some features, their existence matters more  instead of there value
col_with_zero=[] #numeric features with zero values

for i in df_tr.columns:
    if df_tr[i].dtype!='O' and (df_tr[i]==0).any():
        col_with_zero.append(i)

col_with_nan=[] #categorical features with nan values
for i in df_tr.columns:
    if df_tr[i].dtype=='O' and df_tr[i].isna().sum()!=0:
        col_with_nan.append(i)
        


In [159]:
df_tr['is_fence']=df_tr['Fence'].apply(lambda x: 1 if x==x else 0)
df_te['is_fence']=df_te['Fence'].apply(lambda x: 1 if x==x else 0)

#as most of values are missing so dropping fence column




df_tr['is_Pool']=df_tr['PoolQC'].apply(lambda x: 1 if x==x else 0)
df_te['is_Pool']=df_te['PoolQC'].apply(lambda x: 1 if x==x else 0)

df_tr['is_Garage']=df_tr['GarageType'].apply(lambda x: 1 if x==x else 0 )
df_te['is_Garage']=df_te['GarageType'].apply(lambda x: 1 if x==x else 0 )

df_tr['is_Bsmt']=df_tr['BsmtQual'].apply(lambda x: 1 if x==x else 0 )
df_te['is_Bsmt']=df_te['BsmtQual'].apply(lambda x: 1 if x==x else 0 )

df_tr['is_Alley']=df_tr['Alley'].apply(lambda x: 1 if x==x else 0 )

df_te['is_Alley']=df_te['Alley'].apply(lambda x: 1 if x==x else 0 )



In [160]:
#now creating categorical features with the help of numerical features




df_tr['is_Fireplace']=df_tr['Fireplaces'].apply(lambda x: 1 if x>0 else 0)
df_te['is_Fireplace']=df_te['Fireplaces'].apply(lambda x: 1 if x>0 else 0)

df_tr['Tot_BsmtBath']=df_tr['BsmtFullBath']+0.5*df_tr['BsmtHalfBath']
df_te['Tot_BsmtBath']=df_te['BsmtFullBath']+0.5*df_te['BsmtHalfBath']








# Dealing with missing values

In [161]:
for col in df_tr.columns:
    #For categorical features we replace with mode of column or no_col_name
    if df_tr[col].dtype=='O':
        unique_val=dict(df_tr[col].value_counts())
        #if missing values are more than or eqaul to count of a category the replace with no_col_name
        if df_tr[col].isna().sum()>=min(list(unique_val.values())):
            df_tr[col]=df_tr[col].replace(np.nan,'no'+col)
            df_te[col]=df_te[col].replace(np.nan,'no'+col)
        else:
             df_tr[col]=df_tr[col].replace(np.nan,str(df_tr[col].mode()))
             df_te[col]=df_te[col].replace(np.nan,str(df_te[col].mode()))
    elif df_tr[col].dtype!='O' and col!='SalePrice':
        df_tr[col]=df_tr[col].replace(np.nan,df_tr[col].median())

        df_te[col]=df_te[col].replace(np.nan,df_te[col].median())
        
            
        

In [162]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()




In [163]:
df_tr['Alley']=df_tr['Alley'].apply(lambda x:10 if x=='Grvl' else(5 if x=='Pave' else 0))
df_te['Alley']=df_te['Alley'].apply(lambda x:10 if x=='Grvl' else(5 if x=='Pave' else 0))

df_tr['Utilities']=df_tr['Utilities'].apply(lambda x: 10 if x=='AllPub' else(7 if x=='NoSewr' 
                                                                             else(3 if x=='NoSeWa' else 1)))

df_te['Utilities']=df_te['Utilities'].apply(lambda x: 10 if x=='AllPub' else(7 if x=='NoSewr' 
                                                                             else(3 if x=='NoSeWa' else 1)))


df_tr['LandSlope']=df_tr['LandSlope'].apply(lambda x:10 if x=='Gtl' else(5 if x=='Mod' else 2))        

df_te['LandSlope']=df_te['LandSlope'].apply(lambda x:10 if x=='Gtl' else(5 if x=='Mod' else 2))



df_tr['BldgType']=df_tr['BldgType'].apply(lambda x: 2 if x=='1Fam' or x=='TwnhsI' else 1)
df_te['BldgType']=df_te['BldgType'].apply(lambda x: 2 if x=='1Fam' or x=='TwnhsI' else 1)

df_tr['HouseStyle']=df_tr['HouseStyle'].apply(lambda x: 2 if x=='2.5Fin' or x=='2Story' else
                                              (1 if x=='Slvl' or x=='2.5Unf' or x=='1Story' else 0))


df_te['HouseStyle']=df_te['HouseStyle'].apply(lambda x: 2 if x=='2.5Fin' or x=='2Story' else
                                              (1 if x=='Slvl' or x=='2.5Unf' or x=='1Story' else 0))

df_tr['BsmtCond']=df_tr['BsmtCond'].apply(lambda x: 10 if  x=='Gd' else (8 if x=='TA' else( 6 if x=='Fa' else 
                                                                                     0)))

df_te['BsmtCond']=df_te['BsmtCond'].apply(lambda x: 10 if  x=='Gd' else (8 if x=='TA' else( 6 if x=='Fa' else 
                                                                                     0)))

df_tr['BsmtQual']=df_tr['BsmtQual'].apply(lambda x: 10 if  x=='Ex' else (8 if x=='Gd' else( 6 if x=='TA' else 
                                            4 if x=='Fa' else 0)))

df_te['BsmtQual']=df_te['BsmtQual'].apply(lambda x: 10 if  x=='Ex' else (8 if x=='Gd' else( 6 if x=='TA' else 
                                            4 if x=='Fa' else 0)))

df_tr['BsmtExposure']=df_tr['BsmtExposure'].apply(lambda x: 10 if  x=='Ex' else (8 if x=='Gd' else( 6 if x=='Av' else 
                                            4 if x=='Mn' else 0)))

df_te['BsmtExposure']=df_te['BsmtExposure'].apply(lambda x: 10 if  x=='Ex' else (8 if x=='Gd' else( 6 if x=='Av' else 
                                            4 if x=='Mn' else 0)))

df_tr['BsmtFinType1']=df_tr['BsmtFinType1'].apply(lambda x: 10 if  x=='GLQ' else (8 if x=='ALQ' else( 6 if 
                                            x=='BLQ' else 4 if x=='Rec' else (2 if x=='LwQ' else 0))))
df_te['BsmtFinType1']=df_te['BsmtFinType1'].apply(lambda x: 10 if  x=='GLQ' else (8 if x=='ALQ' else( 6 if 
                                            x=='BLQ' else 4 if x=='Rec' else (2 if x=='LwQ' else 0))))

df_tr['BsmtFinType2']=df_tr['BsmtFinType2'].apply(lambda x: 10 if  x=='GLQ' else (8 if x=='ALQ' else( 6 if 
                                            x=='BLQ' else 4 if x=='Rec' else (2 if x=='LwQ' else 0))))
df_te['BsmtFinType2']=df_te['BsmtFinType2'].apply(lambda x: 10 if  x=='GLQ' else (8 if x=='ALQ' else( 6 if 
                                            x=='BLQ' else 4 if x=='Rec' else (2 if x=='LwQ' else 0))))



df_tr['Functional']=df_tr['Functional'].apply(lambda x: 10 if x=='Typ' else
                                              (8 if x=='Min1' else(6 if x=='Min2' else
                                                                   (5 if x=='Mod' else
                                                                    (4 if x=='Maj1' else
                                                                     (3 if x=='Maj2' else(1 if x=='Sev' else 0)))))))

df_te['Functional']=df_te['Functional'].apply(lambda x: 10 if x=='Typ' else
                                              (8 if x=='Min1' else(6 if x=='Min2' else
                                                                   (5 if x=='Mod' else
                                                                    (4 if x=='Maj1' else
                                                                     (3 if x=='Maj2' else(1 if x=='Sev' else 0)))))))


df_tr['FireplaceQu']=df_tr['FireplaceQu'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else 0)))
df_te['FireplaceQu']=df_te['FireplaceQu'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else 0)))

df_tr['PoolQC']=df_tr['PoolQC'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else (3 if x=='Fa' else 0))))

df_te['PoolQC']=df_te['PoolQC'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else (3 if x=='Fa' else 0))))


df_tr['Fence']=df_tr['Fence'].apply(lambda x: 10 if x=='GdPrv' else 
                                    (7 if x=='MnPrv' else (5 if x=='GdWo' else 
                                                           (3 if x=='MnWw' else 0))))
df_te['Fence']=df_te['Fence'].apply(lambda x: 10 if x=='GdPrv' else 
                                    (7 if x=='MnPrv' else (5 if x=='GdWo' else 
                                                           (3 if x=='MnWw' else 0))))



df_tr['ExterQual']=df_tr['ExterQual'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else 0)))
df_te['ExterQual']=df_te['ExterQual'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else 0)))

df_tr['ExterCond']=df_tr['ExterCond'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else 0)))
df_te['ExterCond']=df_te['ExterCond'].apply(lambda x: 10 if x=='Ex' else (8 if x=='Gd' else(
                                                 6 if x=='Ta' else 0)))


df_tr['PavedDrive']=df_tr['PavedDrive'].apply(lambda x: 10 if x=='Y' else(7 if x=='P' else 3))
df_te['PavedDrive']=df_te['PavedDrive'].apply(lambda x: 10 if x=='Y' else(7 if x=='P' else 3))



df_tr['GarageFinish']=df_tr['GarageFinish'].apply(lambda x: 10 if x=='Fin' else 
                                                  (5 if x=='RFn' else 0))
df_te['GarageFinish']=df_te['GarageFinish'].apply(lambda x: 10 if x=='Fin' else 
                                                  (5 if x=='RFn' else 0))

df_tr['GarageCond']=df_tr['GarageCond'].apply(lambda x:10 if x=='Ex' else
                                            (8 if x=='Gd' else(6 if x=='TA' else
                                                               (4 if x=='Fa' else 0))))
df_te['GarageCond']=df_te['GarageCond'].apply(lambda x:10 if x=='Ex' else
                                            (8 if x=='Gd' else(6 if x=='TA' else
                                                               (4 if x=='Fa' else 0))))
df_tr['GarageQual']=df_tr['GarageQual'].apply(lambda x:10 if x=='Ex' else
                                            (8 if x=='Gd' else(6 if x=='TA' else
                                                               (4 if x=='Fa' else 0))))
df_te['GarageQual']=df_te['GarageQual'].apply(lambda x:10 if x=='Ex' else
                                            (8 if x=='Gd' else(6 if x=='TA' else
                                                               (4 if x=='Fa' else 0))))

df_tr['HeatingQC']=df_tr['HeatingQC'].apply(lambda x:10 if x=='Ex' else
                                            (7 if x=='Gd' else(5 if x=='TA' else
                                                               (2 if x=='Fa' else 0))))
df_te['HeatingQC']=df_te['HeatingQC'].apply(lambda x:10 if x=='Ex' else
                                            (7 if x=='Gd' else(5 if x=='TA' else
                                                               (2 if x=='Fa' else 0))))

df_tr['KitchenQual']=df_tr['KitchenQual'].apply(lambda x:10 if x=='Ex' else
                                            (7 if x=='Gd' else(5 if x=='TA' else
                                                               (2 if x=='Fa' else 0))))
df_te['KitchenQual']=df_te['KitchenQual'].apply(lambda x:10 if x=='Ex' else
                                            (7 if x=='Gd' else(5 if x=='TA' else
                                                               (2 if x=='Fa' else 0))))


df_tr['LotShape']=df_tr['LotShape'].apply(lambda x:10 if x=='Reg' else
                                          (7 if x=='IR1' else (2 if x=='IR2' else 0)))

df_te['LotShape']=df_te['LotShape'].apply(lambda x:10 if x=='Reg' else
                                          (7 if x=='IR1' else (2 if x=='IR2' else 0))) 

df_tr['LandContour']=df_tr['LandContour'].apply(lambda x: 10 if x=='Lvl' else
                                                ( 6 if x=='Bnk' else (4 if x=='Low' else 0)))
df_te['LandContour']=df_te['LandContour'].apply(lambda x: 10 if x=='Lvl' else
                                                ( 6 if x=='Bnk' else (4 if x=='Low' else 0)))

In [164]:



# handling rare values in Exterior1st and Exterior2nd 
df_te['Exterior1st']=df_te['Exterior1st'].replace({'0    VinylSd\ndtype: object':'AsphShn'})
df_te['Exterior2nd']=df_te['Exterior2nd'].replace({'0    VinylSd\ndtype: object':'CBlock'})


In [165]:
from scipy.stats import spearmanr
for i in df_tr.columns:
  if df_tr[i].dtype!='O':
    corr,_=spearmanr(df_tr[i],y)
    
    print(i,corr)


# Histplot

In [166]:
import math

        
 

def hist_plot(df,cols,x,y):
    '''df-- dataframe
cols--name of columns
x-- width of graph
y-- height dimension of graph '''
    


            
                            
            
    
    fig =plt.figure(figsize=(x,y))
        
    for index,col in enumerate(cols):
        c=4
        if len(cols)/3==0:
            c=3
        
        plt.subplot(math.ceil(len(cols)/c),c,index+1)
       



        sns.histplot(df[col],color='red',kde=True)
    fig.tight_layout(pad=1.0)
        

        
        
    


   

In [167]:
def numeric(df):
    numeric_des=[] #descrete variables
    numeric_con=[] #continuous variables
    
    for col in df.columns:
        if df[col].dtype=='int64':
            numeric_des.append(col)
        elif df[col].dtype=='float64':
            numeric_con.append(col)
    return (numeric_des,numeric_con)

des,cont=numeric(df_tr)

#most of features are skewed,
#some features which are given as numerical,they should be treated as categorical instead of numerical like MoSold

In [168]:
#for descrete variables
#hist_plot(df_tr,des,30,40)

In [169]:
#continuous variables

#hist_plot(df_tr,cont,30,10)


# count plot 

In [170]:
import math
import warnings
def count_plot(df,x,y):
    
    '''df--DataFrame
    x-- width of graph
    y-- height of graph'''
    categorical=[]
    for col in df.columns:
        if df[col].dtype=='O':
            categorical.append(col)
    
    fig =plt.figure(figsize=(x,y))
    warnings.filterwarnings("ignore")
        
    for index,col in enumerate(categorical):
        
        plt.subplot(math.ceil(len(categorical)/3),3,index+1)
       



        sns.countplot(df[col])
    fig.tight_layout(pad=1.0)
        

In [171]:
#count_plot(df_tr,40,60)

# boxplot

In [172]:
import math
import warnings
def box_plot(df,x,y):
    '''
    df--dataframe
    x-- width of graph
    y-- height of graph'''
    
    numeric=[]
    for col in df.columns:
        if df[col].dtype!='O':
            numeric.append(col)
    
    fig =plt.figure(figsize=(x,y))
    warnings.filterwarnings("ignore")
        
    for index,col in enumerate(numeric):
        
        plt.subplot(math.ceil(len(numeric)/3),3,index+1)
       



        sns.boxplot(y=df[col])
    fig.tight_layout(pad=1.0)
        

In [173]:
#box_plot(df_tr,30,40)

# heatmap

In [174]:
corr = df_train.corr()
fig, ax = plt.subplots(figsize=(40, 15))
#sns.heatmap(corr, vmax=1, square=True,annot=True, fmt='.1f',cmap='RdYlGn');

# regression plot

In [175]:
import math
import warnings
def reg_plot(df,w,h):
    '''w--width of graph
    h--height of graph
    '''
    numeric=[]
    for col in df.columns:
        if df[col].dtype!='O':
            numeric.append(col)
    
    fig =plt.figure(figsize=(w,h))
    warnings.filterwarnings("ignore")
        
    for index,col in enumerate(numeric):
        
        plt.subplot(math.ceil(len(numeric)/3),3,index+1)
       



        sns.regplot(x=df[col],y=y,color='green')
    fig.tight_layout(pad=1.0)

In [176]:
#reg_plot(df_tr,30,90)

# lineplot

In [177]:
import math
import warnings
def line_plot(df,w,h):
    numeric=[]
    for col in df.columns:
        if col=='YearBuilt' or col=='YearRemodAdd' or col=='YrSold':
            numeric.append(col)
    
    fig =plt.figure(figsize=(w,h))
    warnings.filterwarnings("ignore")
        
    for index,col in enumerate(numeric):
        
        plt.subplot(math.ceil(len(numeric)/1),1,index+1)
       



        sns.lineplot(x=df[col],y=y,color='orange', marker='o', markersize=12)
    fig.tight_layout(pad=1.0)

In [178]:
#line_plot(df_tr,20,20)

In [179]:
#there are some features,which make significant change in price only after certain value 
#like house built in 1990 and 1991 will have almost same price if remaining features for both the houses are same

#so applying binning on these features

#in this we are considering that after 5 years of gap price changes significantly.


''' 

step=0
l=list(df_tr['YearBuilt'])
d={}

min_val=min(l)   
while len(l)!=0 and min_val<=max(l):
    j=0
    
   
    
    while j<len(l):
        
        if abs(l[j]-min_val)<=5:
            
            d[l[j]]=step
            
            l.remove(l[j])
            j=j-1
        j=j+1
           
          
    step=step+5
    min_val=min_val+5
               

#now replacing with new values
df_tr['YearBuilt']=df_tr['YearBuilt'].map(d)
df_te['YearBuilt']=df_te['YearBuilt'].map(d)


'''



In [180]:
#now for YearRemodAdd


'''  
step=0
l=list(df_tr['YearRemodAdd'])
d={}

min_val=min(l)   
while len(l)!=0 and min_val<=max(l):
    j=0
    
   
    
    while j<len(l):
        
        if abs(l[j]-min_val)<=5:
            
            d[l[j]]=step
            
            l.remove(l[j])
            j=j-1
        j=j+1
           
          
    step=step+5
    min_val=min_val+5
               

#now replacing with new values
df_tr['YearRemodAdd']=df_tr['YearRemodAdd'].map(d)
df_te['YearRemodAdd']=df_te['YearRemodAdd'].map(d)

'''



In [181]:
#bin width =500

'''

step=500
l=list(df_tr['LotArea'])
d={}

min_val=min(l)   
while len(l)!=0 and min_val<=max(l):
    j=0
    
   
    
    while j<len(l):
        
        if abs(l[j]-min_val)<=500:
            
            d[l[j]]=step
            
            l.remove(l[j])
            j=j-1
        j=j+1
           
          
    step=step+500
    min_val=min_val+500
               

#now replacing with new values
df_tr['LotArea']=df_tr['LotArea'].map(d)
df_te['LotArea']=df_te['LotArea'].map(d)

'''


In [182]:
from scipy.stats import pearsonr
corr,_=pearsonr(df_tr['GarageArea'],y)
corr

In [183]:
# skewness in range of -1 to 1 is tolerable and beyond this data is considered as highly skewed
import scipy

def skewness(df_tr,df_te):
    numeric=[]# list of numeric features
    for col in df_tr.columns:
        if df_tr[col].dtype!='O':
            numeric.append(col)
    
    for col in numeric:
        if scipy.stats.skew(df_tr[col])<-1 or scipy.stats.skew(df_tr[col])>1:
           
            log_skew=scipy.stats.skew(np.log(df_tr[col]+1))
            sqrt_skew=scipy.stats.skew(np.sqrt(df_tr[col]))
            if abs(log_skew)< abs(sqrt_skew) and abs(log_skew)<abs(scipy.stats.skew(df_tr[col])) and col!='SalePrice':
                df_tr[col]=np.log(df_tr[col]+1)
                df_te[col]=np.log(df_te[col]+1)
            elif abs(sqrt_skew)<abs(scipy.stats.skew(df_tr[col])) and col!='SalePrice':
                df_tr[col]=np.sqrt(df_tr[col])
                df_te[col]=np.sqrt(df_te[col])
    return (df_tr,df_te)
    
    

In [184]:
print('Transforming...')
skewness(df_tr,df_te)

print('Transformation Completed')

In [185]:
#as price of house will not change significantly every year ,so let us consider years within a window of (some  years )  as same

#concating the training and test set so same binning is applied for both

'''df=pd.concat([df_tr,df_te])
df.index=[x for x in range(df.shape[0])]

#dividing total data into 10 parts i.e 10bins

df['YearBuilt'] = pd.qcut(df_tr['YearBuilt'], 10,duplicates='drop')



df['YearRemodAdd'] = pd.qcut(df['YearRemodAdd'], 10,duplicates='drop')

df_tr['YearBuilt']=df['YearBuilt'][0:1460]
df_te['YearBuilt']=df['YearBuilt'][1460:]

df_tr['YearRemodAdd']=df['YearRemodAdd'][0:1460]
df_te['YearRemodAdd']=df['YearRemodAdd'][1460:]'''

# feature selection

In [186]:
from sklearn.model_selection import GridSearchCV,KFold,cross_val_score
from sklearn.linear_model import LassoCV,RidgeCV,ElasticNetCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error,r2_score

from scipy.stats import pearsonr

In [187]:

df_tr=df_tr.drop(['BsmtFinSF2','MiscVal',
                    'BsmtHalfBath','HalfBath','LowQualFinSF','MSSubClass',
                    '3SsnPorch','BsmtFullBath'],axis=1)
df_te=df_te.drop(['BsmtFinSF2','MiscVal',
                    'BsmtHalfBath','HalfBath','LowQualFinSF','MSSubClass',
                    '3SsnPorch','BsmtFullBath'],axis=1)

In [188]:
# converting MoSold and YrSold to categorical type
df_tr['MoSold']=df_tr['MoSold'].astype('O')
df_tr['YrSold']=df_tr['YrSold'].astype('O')

df_te['MoSold']=df_te['MoSold'].astype('O')
df_te['YrSold']=df_te['YrSold'].astype('O')

In [189]:
num_features=[]
for i in df_tr.columns:
  if df_tr[i].dtype!='O' and str(i)!='SalePrice':
    corr,_=spearmanr(df_tr[i],y)
    
    num_features.append(i)
    print(i,corr)
  

In [190]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFECV
cat=[]
df_tr1=pd.DataFrame()
for i in df_tr.columns:
  if df_tr[i].dtype=='O':
    cat.append(i)
    le=LabelEncoder()
    df_tr1[i]=le.fit_transform(df_tr[i].values)
    corr,_=spearmanr(df_tr1[i],y)
    print(i,corr)

In [191]:
rfecv=RFECV(RandomForestRegressor (random_state=0))
rfecv.fit(df_tr1.loc[:,cat].values,y)
l=rfecv.get_support().tolist()

cat_features=[]
for i in range(len(l)):
  if l[i]==True:
    corr,_=spearmanr(df_tr1[cat[i]],y)
    print(cat[i],corr)
    
    
    
    cat_features.append(cat[i])

In [192]:
# dropping some of the categorical features
'''
cat_features.remove('MasVnrType')
cat_features.remove('OverallFin')
cat_features.remove('SaleType')

cat_features.remove('Street')
cat_features.remove('LotConfig')


'''

In [193]:
all_features=[]
for i in num_features:
  all_features.append(i)

for i in cat_features:
  all_features.append(i)

In [194]:
len(all_features)

In [195]:
from sklearn.feature_selection import SelectFromModel,RFECV

from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor




In [196]:

# a contains positional values corresponding to each column


In [197]:
a=all_features


In [198]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler


In [199]:
pca=PCA(n_components=20)
sc=StandardScaler()
x_tr=sc.fit_transform(df_tr[num_features].values)
x_te=sc.transform(df_te[num_features].values)
pca_tr=pca.fit_transform(x_tr,y)
pca_te=pca.transform(x_te)

In [200]:
pca_tr.shape,pca_te.shape

In [201]:
df_train_tree=pd.DataFrame()
df_test_tree=pd.DataFrame()
for i in range(20):
  df_train_tree['pca'+str(i+1)]=pca_tr[:,i]
  df_test_tree['pca'+str(i+1)]=pca_te[:,i]


In [202]:
df=pd.concat((df_tr[all_features],df_te[all_features]))
df.index=[x for x in range(df.shape[0])]

df=pd.get_dummies(df)

df_tr1=df.iloc[0:1460,]
df_tr2=df_tr1.copy()
df_te1=df.iloc[1460:,]
df_te2=df_te1.copy()

rs=RobustScaler()
x=rs.fit_transform(df_tr2.values)
df_te1=rs.transform(df_te1.values)

In [203]:
sc=StandardScaler()
df_tr_c=sc.fit_transform(df_tr1)
df_tr=pd.DataFrame(df_tr_c,columns=list(df_tr1.columns))
df_te_c=sc.transform(df_te2)
df_te=pd.DataFrame(df_te_c,columns=list(df_te2.columns))

df_train_f=pd.concat((df_train_tree,df_tr),axis=1)
df_test_f=pd.concat((df_test_tree,df_te),axis=1)

In [204]:
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection


In [205]:

#df_test['YearBuilt']=df_test['YearBuilt'].replace(np.nan,df_test['YearBuilt'].median())
#df_test['LotArea']=df_test['LotArea'].replace(np.nan,df_test['LotArea'].median())
#df_train['LotArea']=df_train['LotArea'].replace(np.nan,df_train['LotArea'].median())

#df_test=df_test.drop('LotArea',axis=1)
#df_train=df_train.drop('LotArea',axis=1)

In [227]:
from sklearn.model_selection import StratifiedKFold
df_tr=df_train_f.copy()
df_tr['Fold']=0
df_train_tree['Fold']=0
skf=StratifiedKFold(n_splits=5,random_state=0)

for f,(t,v) in enumerate(skf.split(df_tr.iloc[:,0:-1],y)):
    df_tr.loc[v,'Fold']=f
df_tr['SalePrice']=y

In [228]:
def train_test(k):
  x_tr=df_tr[df_tr['Fold']!=k].iloc[:,0:-2].values
  y_tr=df_tr[df_tr['Fold']!=k].iloc[:,-1].values

  x_te=df_tr[df_tr['Fold']==k].iloc[:,0:-2].values
  y_te=df_tr[df_tr['Fold']==k].iloc[:,-1].values

  return (x_tr,x_te,y_tr,y_te)

In [230]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import GridSearchCV

In [364]:
def tree_model():
  model_dict={}
  model_dict['lasso']=Lasso(random_state=0,alpha=604,max_iter=5000000)

  model_dict['ridge']=Ridge(random_state=0,alpha=200,max_iter=5000000,solver='sag')

  model_dict['elasticnet']=ElasticNet(random_state=0,alpha=.2,l1_ratio=.14,max_iter=5000000)

  model_dict['svr']=SVR(C=500,  kernel= 'linear',max_iter=5000000,degree=2)

  model_dict['dtr']=DecisionTreeRegressor(random_state=0,max_depth= 5,criterion='mse',
                                          max_features= 'auto')

  model_dict['ada']=AdaBoostRegressor(random_state=0,
                                      learning_rate=0.6,n_estimators=100,loss='exponential')

  model_dict['rfr']=RandomForestRegressor(random_state=0,criterion='mse', max_depth= 11,
                                          max_features='auto',n_estimators=100)

  model_dict['gbr']=GradientBoostingRegressor(random_state=0,learning_rate=0.10, loss='ls', 
                                               max_depth= 3,
                                               n_estimators=100)
  
  model_dict['lgbm']=LGBMRegressor(random_state=0,boosting_type='dart',learning_rate=.168,
                                   max_depth=3,n_estimators=500)

  model_dict['xgb']=XGBRegressor(random_state=0,verbosity=0,booster='gbtree',learning_rate=.105,
                                 max_depth=3,n_estimators=500)
  
  return model_dict

tree_models=tree_model()



In [363]:
m=XGBRegressor(random_state=0,verbosity=0)
params={'max_depth':[3],'n_estimators':[500],'learning_rate':[.1,.095,.105,.11,.112,.115],'booster':['gbtree']}
gcv=GridSearchCV(m,params,cv=5,n_jobs=-1,verbose=1)
gcv.fit(df_train_f.values,y)

In [366]:
gcv.best_params_

In [362]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
start_time = datetime.now()
# do your work here
end_time = datetime.now()
rmse=[]
start_time = datetime.now()
for i in range(5):
  x_tr,x_te,y_tr,y_te=train_test(i)
  
  

  

  model=tree_models['xgb']
  
  model.fit(x_tr,y_tr)
  y_pr=model.predict(x_te)

  error=np.sqrt(mean_squared_error(y_te,y_pr))
  rmse.append(error)
  print('fold=%d,error=%f'%(i+1,error))
  end_time = datetime.now()
  

print('RMSE for %s model is %f'%('lasso',sum(rmse)/5))
print(f'time taken=',(end_time-start_time))
print()



In [213]:
'''
from datetime import datetime
start_time = datetime.now()
# do your work here
end_time = datetime.now()
rmse=[]
start_time = datetime.now()
for i in range(5):
  x_tr,x_te,y_tr,y_te=tree_train_test(i)
  
  

  rs1=RobustScaler()
  x_tr=rs1.fit_transform(x_tr)
  x_te=rs1.transform(x_te)

  model=tree_models[']
  
  model.fit(x_tr,y_tr)
  y_pr=model.predict(x_te)

  error=np.sqrt(mean_squared_error(y_te,y_pr))
  rmse.append(error)
  print('fold=%d,error=%f'%(i+1,error))
  end_time = datetime.now()
  

print('RMSE for %s model is %f'%('lasso',sum(rmse)/5))
print(f'time taken=',(end_time-start_time))
print()


'''

In [214]:
'''


for k,v in models.items():
  print('Calculating RMSE for %s model'%(k))

  rmse=[]

  for i in range(5):
    start_time = datetime.now()
    x_tr,x_te,y_tr,y_te=tree_train_test(i)
    

    rs1=RobustScaler()
    x_tr=rs1.fit_transform(x_tr)
    x_te=rs1.transform(x_te)

    model=models[k]
    model.fit(x_tr,y_tr)
    y_pr=model.predict(x_te)

    end_time = datetime.now()

    error=np.sqrt(mean_squared_error(y_te,y_pr))
    rmse.append(error)
    
  
  print('RMSE for %s model is %f'%(k,sum(rmse)/5),(end_time-start_time))
  print()
  print()

'''

In [215]:
def tree_train_test(k):
  x_tr=df_train_tree[df_tr1['Fold']!=k].iloc[:,0:-1].values
  y_tr=df_tr1[df_tr1['Fold']!=k].iloc[:,-1].values

  x_te=df_train_tree[df_tr1['Fold']==k].iloc[:,0:-1].values
  y_te=df_tr1[df_tr1['Fold']==k].iloc[:,-1].values

  return (x_tr,x_te,y_tr,y_te)
  return (x_tr,x_te,y_tr,y_te)

In [216]:
'''


for k,v in tree_models.items():
  print('Calculating RMSE for %s model'%(k))

  rmse=[]

  for i in range(5):
    start_time = datetime.now()
    x_tr,x_te,y_tr,y_te=tree_train_test(i)
    

    

    model=tree_models[k]
    model.fit(x_tr,y_tr)
    y_pr=model.predict(x_te)

    end_time = datetime.now()

    error=np.sqrt(mean_squared_error(y_te,y_pr))
    rmse.append(error)
    
  
  print('RMSE for %s model is %f'%(k,sum(rmse)/5),(end_time-start_time))
  print()
  print()

'''

In [217]:
'''

s = datetime.now()
df_train_tree=df_train_tree.drop(['Fold'],axis=1)

df_tr1=df_tr1.drop(['Fold','SalePrice'],axis=1)

df_tr1_copy=df_tr1.copy()
rs=StandardScaler()
df_tr1=pd.DataFrame(rs.fit_transform(df_tr1),columns=list(df_tr1_copy.columns))
df_te2_copy=df_te2.copy()
df_te2=pd.DataFrame(rs.transform(df_te2),columns=list(df_te2_copy.columns))


df_test_result=pd.DataFrame()
df_train_result=pd.DataFrame()

'''



In [218]:
'''
for k in list(models.keys()):
    
    m=models[k].fit(df_tr1.values,y)
    y_pr=m.predict(df_tr1.values)
    
    
    df_train_result[k]=y_pr
    y_pr=m.predict(df_te2.values)
    df_test_result[k]=y_pr

for k in list(tree_models.keys()):
    
    m=tree_models[k].fit(df_train_tree.values,y)
    y_pr=m.predict(df_train_tree.values)
    df_train_result[k]=y_pr
    y_pr=m.predict(df_test_tree.values)
    df_test_result[k]=y_pr
    
    
df_train=pd.concat((df_tr1,df_train_result),axis=1)
df_test=pd.concat((df_te2,df_test_result),axis=1)
df_test
m=tree_models['xgb'].fit(df_train_result.values,y)
y_pr=m.predict(df_test_result.values)
e = datetime.now()
print(e-s)
'''

In [371]:
final_model=StackingCVRegressor(regressors=(tree_models['lasso'],tree_models['elasticnet'],tree_models['ridge']
                                            ,tree_models['svr'],tree_models['dtr'],tree_models['ada'],
                                            
                                         
                                          tree_models['lgbm'],tree_models['xgb'],tree_models['rfr'],tree_models['gbr']),
                              meta_regressor=tree_models['rfr'],use_features_in_secondary=True)

In [220]:

'''
import warnings
warnings.filterwarnings('ignore')
rmse=[]
from sklearn.preprocessing import MinMaxScaler
for i in range(5): 

    s = datetime.now()
   
  
    x_tr,x_te,y_tr,y_te=tree_train_test(i)
    

    rs1=RobustScaler()
    #x_tr=rs1.fit_transform(x_tr)
    #x_te=rs1.transform(x_te)

    model=final_model
    
    model.fit(x_tr,y_tr)
    
    
    y_pr=model.predict(x_te)

    error=np.sqrt(mean_squared_error(y_te,y_pr))
    rmse.append(error)

    e = datetime.now()
    

    print(' %d fold completed and error=%f'%(i+1,error))
    print('time taken=',e-s)
 
print('RMSE for %s model is %f'%('stack_model',sum(rmse)/5))

'''


In [372]:

s = datetime.now()
final_model.fit(df_train_f.values,y)
y_pr=final_model.predict(df_test_f.values)
e = datetime.now()
print(e-s)

In [373]:
df_test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

id=df_test['Id']

df_csv=pd.DataFrame({'Id':id,'SalePrice':y_pr})
df_csv.to_csv('submission4.csv',index=False)

In [374]:
df_csv